In [1]:
# -----Installazione di tutti gli strumenti e librerie utili per eseguire il codice-----

!pip install websocket-client
!pip install ibm-cos-sdk
! pip install tweepy
! pip install textblob

import pandas as pd
import json, types
import websocket
import _thread
import time
import ibm_boto3
import codecs
import tweepy 
import numpy as np
import re
import matplotlib.pyplot as plt

from io import StringIO
from textblob import TextBlob


#  -----Twitter API credentials-----
# Modificare le seguenti variabili con le proprie chiavi
consumer_key = "consumer_key"
consumer_secret = "consumer_secret"
access_key = "access_key-w8eY4gMKp1tlH5gTuQXpvwjSQTGYpS"
access_secret = "access_secret"
OAUTH_KEYS = {'consumer_key':consumer_key, 'consumer_secret':consumer_secret,
 'access_token_key':access_key, 'access_token_secret':access_secret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)


In [2]:
# # # # TWITTER CLIENT # # # #
#Classe i cui attributi sono API Twitter, le credenziali dell'autenticazione
#il nome dell'utente
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = auth
        self.twitter_client = api

        self.twitter_user = twitter_user
    
    #Return twetter api
    def get_twitter_client_api(self):
        return self.twitter_client
    
    #Returns the num_tweets most recent statuses by specific user, including retweets, posted by the authenticating user and that user’s friends.
    def get_user_timeline_tweets(self, num_tweets,hashtag):
        tweets = []
        for tweet in tweepy.Cursor(self.twitter_client.user_timeline, q=hashtag,id=self.twitter_user, tweet_mode='extended').items(num_tweets):
            tweets.append(tweet)
            print(tweet)
        return tweets
    
     #Returns the num_tweets most recent statuses by specific user, including retweets, posted by the authenticating user and that user’s friends.
    def get_hashtag_timeline_tweets(self, num_tweets,hashtag):
        tweets = []
        for tweet in tweepy.Cursor(self.twitter_client.search, q=hashtag, tweet_mode='extended', include_rts = False).items(num_tweets):
            tweets.append(tweet)
            print(tweet)
        return tweets


In [3]:
#classe che permette di raccogliere e analizzare ed effettuare il mining dei tweet
#a cui viene passato un oggetto tweet client
class TweetAnalyzer():
    def __init__(self, twitter_client):
        self.twitter_client = twitter_client
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    #metodo che rimuove i informazioni non necessarie e pulisce il tweet da caratteri non utili
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    #metodo per eseguire la sentiment analyzis
    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return 1
        elif analysis.sentiment.polarity == 0:
            return 0
        else:
            return -1
    
    #All possible attribute
    """
    ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', 
    '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', 
    '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_api', 
    '_json', 'author', 'contributors', 'coordinates', 'created_at', 'destroy', 'entities', 'favorite', 'favorite_count', 'favorited', 
    'geo', 'id', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 
    'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'parse', 'parse_list', 'place', 'possibly_sensitive', 'retweet', 'retweet_count', 
    'retweeted', 'retweets', 'source', 'source_url', 'text', 'truncated', 'user']
    """
    #metodo per trasformare i tweet dal formato fornito al froamto dataframe
    def tweets_to_data_frame(self, tweets):
        #df = pd.DataFrame(data=[tweet.full_text for tweet in tweets], columns=['tweets'])
        
        result_t = []
        for tweet in tweets:
            try:
                result_t += [tweet.retweeted_status.full_text]
            except AttributeError: # Not a Retweet
                result_t += [tweet.full_text]

            
        #Creato un array che rappresentano le varie colone della tabella con i tweet raccolti
        
        df = pd.DataFrame(data=result_t, columns=['tweets'])
        #id tweet
        df['id'] = np.array([tweet.id for tweet in tweets])
        #length of text
        df['len'] = np.array([len(tweet.full_text) for tweet in tweets])
        #data tweet
        df['date'] = np.array([tweet.created_at for tweet in tweets])
        #sorgente del tweet
        df['source'] = np.array([tweet.source for tweet in tweets])
        #numero di like del tweet
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        #number of retweets
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])
        
        #All possible attribute of a user
        """
        'created_at', 'default_profile', 'default_profile_image', 'description', 'entities', 'favourites_count', 'follow', 
        'follow_request_sent', 'followers', 'followers_count', 'followers_ids', 'following', 'friends', 'friends_count', 'geo_enabled', 
        'has_extended_profile', 'id', 'id_str', 'is_translation_enabled', 'is_translator', 'lang', 'listed_count', 'lists', 
        'lists_memberships', 'lists_subscriptions', 'location', 'name'
        """
         
        user_data = [tweet.user for tweet in tweets]
        #nome utente di cui si esegue la ricerca
        df['name'] = np.array([user_tweet.name for user_tweet in user_data])
        #descrizione profilo utente cercato
        df['description'] = np.array([user_tweet.description for user_tweet in user_data])
        #location del tweet in cui è stato fatto
        df['location'] = np.array([user_tweet.location for user_tweet in user_data])
        #data in cui è stato creato l'account
        df['created_at'] = np.array([user_tweet.created_at for user_tweet in user_data])
        #numero di follower del profilo
        df['followers_count'] = np.array([user_tweet.followers_count for user_tweet in user_data])
        df['friends_count'] = np.array([user_tweet.friends_count for user_tweet in user_data])
        #booleano per indicare se si segue o meno tale profilo
        df['following'] = np.array([user_tweet.following for user_tweet in user_data])
            
            

        return df

In [4]:
#Metodo del websocket che permette di ricevere le richieste da front-end e a seconda della richiesta
#elabora una risposta adeguata alla richiesta
#---Sono presenti 2 comandi "MINING" e "MININGHASHTAG"
#Il primo per eseguire la ricerca su di uno specifico utente 
#La seconda su tweet con un particolare hashtag o parola chiave
def on_message(ws, message):
    print(message)
    msg = json.loads(message)
    cmd = msg['cmd']
    if cmd == 'MINING':
        user = msg['user']
        
        numTweet = int(msg['numTweet'])
        hashtag = msg['hashtag']
        twitter_client = TwitterClient(user)
        tweet_analyzer = TweetAnalyzer(twitter_client)
        #Tweets_filter
        tweets = twitter_client.get_user_timeline_tweets(numTweet,hashtag)

        #From json to DataFrame
        df = tweet_analyzer.tweets_to_data_frame(tweets)

        #Sentiment Analysis
        df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
        out = df.to_json(orient='records')
        #print(df.head(numTweet))
        responsejson = {}
        responsejson['forcmd']='MINING'
        responsejson['response']= out
        ws.send(json.dumps(responsejson))
    if cmd == 'MININGHASHTAG':
        numTweet = int(msg['numTweet'])
        hashtag = msg['hashtag']
        twitter_client = TwitterClient()
        tweet_analyzer = TweetAnalyzer(twitter_client)
        #Tweets_filter
        tweets = twitter_client.get_hashtag_timeline_tweets(numTweet,hashtag)

        #From json to DataFrame
        df = tweet_analyzer.tweets_to_data_frame(tweets)

        #Sentiment Analysis
        df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])
        out = df.to_json(orient='records')
        #print(df.head(numTweet))
        responsejson = {}
        responsejson['forcmd']='MININGHASHTAG'
        responsejson['response']= out
        ws.send(json.dumps(responsejson))

#Metodi per gestire gli errori
def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("MiningTwitter BackEnd Closed")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"MiningTwitter BackEnd Connected"}'
            ws.send(hbeat)
            time.sleep(100)
            
    _thread.start_new_thread(run, ())

# Metodo che apre websocket in ascolto e si collega 
def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://Node-Red-Url",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

In [ ]:
#Chiamo la funzione start_websocket_listener e mi metto in ascolto
start_websocket_listener()

--- request header ---
GET /ws/orchestrate HTTP/1.1
Upgrade: websocket
Host: node-red-kwxyx-2020-08-25.eu-gb.mybluemix.net
Origin: http://node-red-kwxyx-2020-08-25.eu-gb.mybluemix.net
Sec-WebSocket-Key: GRT1pHmrmEqFpbJYz/puuQ==
Sec-WebSocket-Version: 13
Connection: upgrade


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Connection: Upgrade
Sec-WebSocket-Accept: UxUGUqCV6TBrWE9Yeo3WurTUYsw=
Date: Thu, 03 Sep 2020 13:19:58 GMT
X-Global-Transaction-ID: 5ed21c225f50ed7eed0339cd
Upgrade: websocket
-----------------------
send: b"\x81\xa9*\x07\xef\xafQ%\x8c\xc2N%\xd5\x8dgn\x81\xc6D`\xbb\xd8Cs\x9b\xcaX'\xad\xceIl\xaa\xc1N'\xac\xc0Di\x8a\xcc^b\x8b\x8dW"


{"cmd":"MiningTwitter BackEnd Connected"}


send: b'\x81\xa9\xd59\x88\x82\xae\x1b\xeb\xef\xb1\x1b\xb2\xa0\x98P\xe6\xeb\xbb^\xdc\xf5\xbcM\xfc\xe7\xa7\x19\xca\xe3\xb6R\xcd\xec\xb1\x19\xcb\xed\xbbW\xed\xe1\xa1\\\xec\xa0\xa8'


{"cmd":"MiningTwitter BackEnd Connected"}


send: b'\x81\xa9\xacZ\xc3\xf6\xd7x\xa0\x9b\xc8x\xf9\xd4\xe13\xad\x9f\xc2=\x97\x81\xc5.\xb7\x93\xdez\x81\x97\xcf1\x86\x98\xc8z\x80\x99\xc24\xa6\x95\xd8?\xa7\xd4\xd1'


...

{"cmd":"MiningTwitter BackEnd Connected"}


send: b'\x81\xa9\x86C*S\xfdaI>\xe2a\x10q\xcb*D:\xe8$~$\xef7^6\xf4ch2\xe5(o=\xe2ci<\xe8-O0\xf2&Nq\xfb'


{"cmd":"MiningTwitter BackEnd Connected"}


send: b'\x81\xa9nA\x8b\xef\x15c\xe8\x82\nc\xb1\xcd#(\xe5\x86\x00&\xdf\x98\x075\xff\x8a\x1ca\xc9\x8e\r*\xce\x81\na\xc8\x80\x00/\xee\x8c\x1a$\xef\xcd\x13'


{"cmd":"MiningTwitter BackEnd Connected"}
